In [ ]:
import sys
sys.path.append('../')

In [ ]:
from src.datasets import TileTripletsDataset, GetBands, RandomFlipAndRotate, ClipAndScale, ToFloatTensor, triplet_dataloader

# Step 1. Download triplets from bucket

Using the download link, unzip triplets into the directory tile2vec/data/naip.

# Step 2. Train TileNet

In [ ]:
# Environment stuff
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
cuda = torch.cuda.is_available()

Set up the dataloader for training.

In [ ]:
# Change these arguments to match your directory and desired parameters
patch_dir = '/atlas/u/nj/GitHub/pixel2vec/data/interim/naip_fresno/triplets50_100/'
augment = True
batch_size = args.batch_size
shuffle = True
num_workers = 4
n_triplets = args.n_triplets

In [ ]:
dataloader = triplet_dataloader(img_type, patch_dir, bands=bands,
    augment=augment, batch_size=batch_size, shuffle=shuffle,
    num_workers=num_workers, n_triplets=n_triplets, normalize=normalize)
print('Dataloader set up complete.')

Set up the CNN.

In [ ]:
in_channels = bands
supervised = False
no_relu = False
patch_size = 50
n_filters = 50
kernel_size = 3

In [ ]:
if model == 'resnet':
    cnn = ResNet18(in_channels=in_channels, z_dim=z_dim,
        supervised=supervised, no_relu=no_relu, loss_type=loss_type,
        activation=activation)
elif model == 'cnn':
    cnn = CNN(in_channels=in_channels, z_dim=z_dim, patch_size=patch_size,
        n_filters=n_filters, kernel_size=kernel_size)
cnn.train()
if cuda: cnn.cuda()
print('CNN set up complete.')

Set up optimizer.

In [ ]:
lr = 1e-3
optimizer = optim.Adam(cnn.parameters(), lr=lr, betas=(0.5, 0.999))

Directory for saving models.

In [ ]:
model_dir = os.path.join('/atlas/u/nj/GitHub/pixel2vec/models/', env)
if not os.path.exists(model_dir): os.makedirs(model_dir)
config_fn = os.path.join(model_dir, 'config.yaml')
config = vars(args)
config['env'] = env
if save_models:
    with open(config_fn, 'w') as file:
        print('Writing config file')
        file.write(dump(config))

Train model!

In [ ]:
t0 = time()
results_fn = os.path.join(model_dir, 'results.txt')
with open(results_fn, 'w') as file:

    print('Begin training.................')
    for epoch in range(0, epochs):
        (avg_loss, avg_l_n, avg_l_d, avg_l_nd) = train_triplet_epoch(
            cnn, cuda, dataloader, optimizer, epoch+1, margin=margin, l2=l2,
            print_every=print_every, t0=t0)
#         vis_data['avg_losses'].append(avg_loss)
#         vis_data['avg_l_ns'].append(avg_l_n)
#         vis_data['avg_l_ds'].append(avg_l_d)
#         vis_data['avg_l_nds'].append(avg_l_nd)

        # Plot loss
#         Y = np.array(vis_data['avg_losses'])
#         vis.line(Y=Y, X=np.array(range(1, epoch+2)), win='loss',
#             opts={'legend': ['loss'], 'markers': False,
#             'title': 'Training loss',
#             'xlabel': 'Epoch', 'ylabel': 'Average loss'})

        # Plot l_n, l_d, and l_nd if available
        if loss_type == 'triplet':
            Y = np.vstack((np.array(vis_data['avg_l_ns']),
                           np.array(vis_data['avg_l_ds']),
                           np.array(vis_data['avg_l_nds']))).T
            legend = ['l_n', 'l_d', 'l_nd']
        elif loss_type == 'cosine':
            Y = np.vstack((np.array(vis_data['avg_l_ns']),
                           np.array(vis_data['avg_l_ds']))).T
            legend = ['l_n', 'l_d']
        vis.line(Y=Y, X=np.array(range(1, epoch+2)), win='losses',
            opts={'legend': legend, 'markers': False,
            'title': 'Loss components', 'xlabel': 'Epoch',
            'ylabel': 'Average loss'})

        # RF comparison with PCA
        X = embed_dataset(cnn, z_dim, cuda, img_type, patch_dir,
            bands=bands, augment=False, batch_size=50,
            shuffle=False, num_workers=4, print_every=None,
            n_triplets=rf_triplets)
        vis_data['avg_z_norms'].append(np.linalg.norm(X, axis=1).mean())
        X = np.delete(X, nan_locs, axis=0)
        X_tr, X_te, X_tr_pca, X_te_pca, y_tr, y_te = train_test_split(X, X_pca, y, shuffle=True)
        rf = RandomForestClassifier()
        rf.fit(X_tr, y_tr)
        rf_acc = rf.score(X_te, y_te)
        rf_accs['patch2vec'].append(rf_acc)
        rf_accs['PCA'].append(rf_accs['PCA'][-1])
        Y = np.vstack((np.array(rf_accs['patch2vec']), rf_accs['PCA'])).T
        vis.line(Y=Y, X=np.array(range(0, epoch+2)), win='rf',
            opts={'legend': ['patch2vec', 'PCA'], 'markers': False,
            'title': 'RF comparison on CDL', 'xlabel': 'Epoch',
            'ylabel': 'Accuracy'})

        print('Writing results for epoch {}'.format(epoch+1))
        file.write('{} {} {} {} {}\n'.format(
            avg_loss, avg_l_n, avg_l_d, avg_l_nd, rf_acc))

        # Plot average norm for embedding
        Y = np.array(vis_data['avg_z_norms'])
        vis.line(Y=Y, X=np.array(range(0, epoch+2)), win='z_norm',
            opts={'title': 'Average embedding norm', 'xlabel': 'Epoch',
            'ylabel': 'Norm'})

        # Plotting PCA of embeddings by CDL class
        X_embed_pca = PCA(2).fit_transform(X)
        vis.scatter(X=X_embed_pca[:1000], Y=y[:1000], win='pca{}'.format(epoch % 3),
            opts={'title': 'Epoch {}: PCA by CDL label'.format(epoch+1),
            'xlabel': 'PC 1', 'ylabel': 'PC 2'})

        # Plot histograms of latent dimensions
        for d_z in range(min(z_dim, 3)):
            vis.histogram(X=X[:,d_z], win='z{}'.format(d_z),
                opts={'numbins': 20, 'title': 'Epoch {}: Distribution of z{}'.format(epoch+1, d_z)})

        # Plot nearest neighbors for embedding and for PCA
        n_samples = 5
        nrow = 8
        k = int(nrow ** 2)
        for i in range(n_samples):
            samples = np.zeros((k, 3, patch_size, patch_size))
            idx = np.random.randint(0, len(patches))
            # Get embedding and PCA neighbors
            (topk_idxs, topk_dists) = get_k_neighbors(idx, X, int(k/2))
            (topk_idxs_pca, topk_dists_pca) = get_k_neighbors(idx, X_pca, int(k/2))
            X_idx = 0
            X_pca_idx = 0
            for j in range(k):
                if j % nrow < nrow / 2:
                    topk_idx = topk_idxs[X_idx]
                    X_idx += 1
                else:
                    topk_idx = topk_idxs_pca[X_pca_idx]
                    X_pca_idx += 1
                samples[j,:,:,:] = np.moveaxis(patches[topk_idx][:,:,:3], -1, 0)
            vis.images(samples, win='knn{}_{}'.format(epoch % 5, i),
                opts={'nrow': nrow, 'title': '=== patch2vec (left half) ===== Epoch {}, #{} ===== PCA (right half) =========='.format(epoch+1, i+1)})

        # Save model
        if save_models:
            if rf_acc == np.max(np.array(rf_accs['patch2vec'])) or epoch == epochs-1:
                model_fn = os.path.join(model_dir, 'epoch{}.ckpt'.format(epoch+1))
                torch.save(cnn.state_dict(), model_fn)